# Scientific Paper Agent

In this notebook, we will create an agent that given a dataframe will answer user questions about it. This example is based on the [Scientific Paper Agent](https://github.com/NirDiamant/GenAI_Agents/blob/main/all_agents_tutorials/scientific_paper_agent_langgraph.ipynb) from the [GenAI Agents](https://github.com/NirDiamant/GenAI_Agents) repository.

In [1]:
import os 
import io
import urllib3
# import pdfplumber
import time

from datetime import datetime, timedelta
import requests
from pydantic import BaseModel, Field
from typing import Annotated, ClassVar

from agente.core.base import BaseAgent,BaseTaskAgent
from agente.core.decorators import function_tool,agent_tool

## Load and set environment variables from .env file
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from dotenv import load_dotenv
load_dotenv()

True

### First, let's define the Core API Wrapper

In [2]:
class CoreAPIWrapper(BaseModel):
    """Simple wrapper around the CORE API."""
    base_url: ClassVar[str] = "https://api.core.ac.uk/v3"
    api_key: ClassVar[str] = os.environ["CORE_API_KEY"]

    top_k_results: int = Field(description = "Top k results obtained by running a query on Core", default = 1)

    def _get_search_response(self, query: str) -> dict:
        http = urllib3.PoolManager()

        # Retry mechanism to handle transient errors
        max_retries = 5    
        for attempt in range(max_retries):
            response = http.request(
                'GET',
                f"{self.base_url}/search/outputs", 
                headers={"Authorization": f"Bearer {self.api_key}"}, 
                fields={"q": query, "limit": self.top_k_results}
            )
            if 200 <= response.status < 300:
                return response.json()
            elif attempt < max_retries - 1:
                time.sleep(2 ** (attempt + 2))
            else:
                raise Exception(f"Got non 2xx response from CORE API: {response.status} {response.data}")

    def search(self, query: str) -> str:
        response = self._get_search_response(query)
        results = response.get("results", [])
        if not results:
            return "No relevant results were found"

        # Format the results in a string
        docs = []
        for result in results:
            published_date_str = result.get('publishedDate') or result.get('yearPublished', '')
            authors_str = ' and '.join([item['name'] for item in result.get('authors', [])])
            docs.append((
                f"* ID: {result.get('id', '')},\n"
                f"* Title: {result.get('title', '')},\n"
                f"* Published Date: {published_date_str},\n"
                f"* Authors: {authors_str},\n"
                f"* Abstract: {result.get('abstract', '')},\n"
                f"* Paper URLs: {result.get('sourceFulltextUrls') or result.get('downloadUrl', '')}"
            ))
        return "\n-----\n".join(docs)

### Now we start by defining the Judge Adgent, because is the last one to be used in the pipeline

In [3]:
class JudgeAgent(BaseTaskAgent):
    """Agent that judges the quality of the final answer provided by the research agent."""

    agent_name:str = "judge_agent"
    system_prompt: str = """You are an expert scientific researcher.
Your goal is to review the final answer you provided for a specific user query.

Look at the conversation history between you and the user. Based on it, you need to decide if the final answer is satisfactory or not.

A good final answer should:
- Directly answer the user query. For example, it does not answer a question about a different paper or area of research.
- Answer extensively the request from the user.
- Take into account any feedback given through the conversation.
- Provide inline sources to support any claim made in the answer.

In case the answer is not good enough, provide clear and concise feedback on what needs to be improved to pass the evaluation.

Write your evaluation by calling the `complete_task` function with the evaluation as an argument.
"""
    completion_kwargs: dict = {
        "model": "gpt-4.1",
        "stream": True,
    }
    user_query: str = None

    @function_tool
    def task_completed(self, 
                        evaluation: Annotated[str, "The evaluation of the final answer."], 
                        re_do: Annotated[bool, "Whether to go repeat the research process or not. Default is False."] = False) -> str:
        """Evaluate the final answer and provide feedback."""
        if re_do:
            self.parent_agent.next_tool_map["judge_agent"] = "planning_agent"

        return evaluation

### Now we define the Planning Agent

In [14]:
class PlanningAgent(BaseTaskAgent):
    """Agent that plans the search for research papers using the CORE API."""

    agent_name:str = "planing_agent"
    system_prompt: str = """# IDENTITY AND PURPOSE

You are an experienced scientific researcher.
Your goal is to make a new step by step plan to help the user with their scientific research .

Subtasks should not rely on any assumptions or guesses, but only rely on the information provided in the context or look up for any additional information.

If any feedback is provided about a previous answer, incorportate it in your new planning.


# TOOLS

For each subtask, indicate the given tool required to complete the subtask. 
Tools can be one of the following:

- search_papers: Search for research papers using the CORE API.
- download_paper: Download a research paper from a given URL.
- ask_human_feedback: Ask for human feedback on a given answer.
"""
    completion_kwargs: dict = {
        "model": "gpt-4.1",
        "stream": True,
    }


    @function_tool
    def task_completed(self, plan: Annotated[str, "The plan to be executed"]) -> str:
        """Complete the task by providing the plan."""
        return plan

### Now the research agent, that will be solely responsible to execute the research plan

In [15]:
class ResearchAgent(BaseTaskAgent):
    """Agent that do the research for the user."""

    agent_name:str = "research_agent"
    system_prompt: str = """# IDENTITY AND PURPOSE

You are an experienced scientific researcher. 
Your goal is to help the user with their scientific research. You have access to a set of external tools to complete your tasks.
Follow the plan given by the planning agent to successfully complete the task.
At the end call the complete_task tool to provide the final answer to the user.

Add extensive inline citations to support any claim made in the answer.


# EXTERNAL KNOWLEDGE

## CORE API

The CORE API has a specific query language that allows you to explore a vast papers collection and perform complex queries. See the following table for a list of available operators:

| Operator       | Accepted symbols         | Meaning                                                                                      |
|---------------|-------------------------|----------------------------------------------------------------------------------------------|
| And           | AND, +, space          | Logical binary and.                                                                           |
| Or            | OR                     | Logical binary or.                                                                            |
| Grouping      | (...)                  | Used to prioritise and group elements of the query.                                           |
| Field lookup  | field_name:value       | Used to support lookup of specific fields.                                                    |
| Range queries | fieldName(>, <,>=, <=) | For numeric and date fields, it allows to specify a range of valid values to return.         |
| Exists queries| _exists_:fieldName     | Allows for complex queries, it returns all the items where the field specified by fieldName is not empty. |

Use this table to formulate more complex queries filtering for specific papers, for example publication date/year.
Here are the relevant fields of a paper object you can use to filter the results:
{
  "authors": [{"name": "Last Name, First Name"}],
  "documentType": "presentation" or "research" or "thesis",
  "publishedDate": "2019-08-24T14:15:22Z",
  "title": "Title of the paper",
  "yearPublished": "2019"
}

Example queries:
- "machine learning AND yearPublished:2023"
- "maritime biology AND yearPublished>=2023 AND yearPublished<=2024"
- "cancer research AND authors:Vaswani, Ashish AND authors:Bello, Irwan"
- "title:Attention is all you need"
- "mathematics AND _exists_:abstract"
"""

    completion_kwargs: dict = {
        "model": "gpt-4.1",
        "stream": True,
        "tool_choice": "required"
    }


    @function_tool
    def search_papers(self,
                        query: Annotated[str, "The query to search for scientific papers."],
                        max_papers: Annotated[int, "The maximum number of papers to return. Default is 1."] = 1) -> str:
        """Search for scientific papers using the CORE API.

        Example:
        {"query": "Attention is all you need", "max_papers": 1}
        """
        try:
            return CoreAPIWrapper(top_k_results=max_papers).search(query)
        except Exception as e:
            return f"Error performing paper search: {e}"

    @function_tool
    def download_paper(self,url: Annotated[str, "The URL to download the scientific paper"]) -> str:
        """Download a specific scientific paper from a given URL.

        Example:
        {"url": "https://sample.pdf"}
        """
        try:        
            http = urllib3.PoolManager(
                cert_reqs='CERT_NONE',
            )
            
            # Mock browser headers to avoid 403 error
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Language': 'en-US,en;q=0.5',
                'Accept-Encoding': 'gzip, deflate, br',
                'Connection': 'keep-alive',
            }
            max_retries = 5
            for attempt in range(max_retries):
                response = http.request('GET', url, headers=headers)
                if 200 <= response.status < 300:
                    pdf_file = io.BytesIO(response.data)
                    with pdfplumber.open(pdf_file) as pdf:
                        text = ""
                        for page in pdf.pages:
                            text += page.extract_text() + "\n"
                    return text
                elif attempt < max_retries - 1:
                    time.sleep(2 ** (attempt + 2))
                else:
                    raise Exception(f"Got non 2xx when downloading paper: {response.status_code} {response.text}")
        except Exception as e:
            return f"Error downloading paper: {e}"


    @agent_tool
    def ask_human_feedback(self,question: Annotated[str, "The question to ask for human feedback"]) -> str:
        """Ask for human feedback. You should call this tool when encountering unexpected errors. """

        class HumanFeedback(BaseTaskAgent):

            agent_name:str = "human_feedback"

            @function_tool
            def complete_task(self, feedback: str) -> str:
                """Complete the task by providing the feedback.

                Args:
                    feedback: The feedback provided by the human.
                """
                return feedback
        
        human_feedback = HumanFeedback()
        human_feedback.add_message("assistant",question)
        return human_feedback
            

    @function_tool
    def task_completed(self, final_answer: Annotated[str, "The final answer to the user query."]) -> str:
        """Complete the task by providing the final answer. """
        return final_answer

### Now let's define the main agent, responsible to orchestrate the other agents. 

Here, we merge the original, and initial, decision step in the same main agent. It will decice if it replies directly or start the research pipeline, by calling the planning agent.

In [16]:
class MainAgent(BaseAgent):

    agent_name:str = "MainAgent"
    system_prompt: str = """You are an experienced scientific researcher.
Your goal is to help the user with their scientific research.

Based on the user query, decide if you need to perform a research or if you can answer the question directly.
- You should perform a research if the user query requires any supporting evidence or information. In this case, you should call first the planning agent to create a plan and then the research agent to execute the plan, and finally the judge agent to evaluate the final answer.
- You should answer the question directly only for simple conversational questions, like "how are you?".
"""    
    completion_kwargs: dict = {
        "model": "gpt-4.1",
        "stream": True
    }

    user_query:str = None

    @agent_tool(next_tool="research_agent",manual_call=lambda m:{'plan':m})
    def planning_agent(self,user_query: str):
        """The query to be planned

        Args:
            user_query: The user query.        
        """
        self.user_query = user_query
        planning_agent = PlanningAgent()
        planning_agent.add_message("user",user_query)
        return planning_agent

    @agent_tool(next_tool="judge_agent",manual_call=lambda m:{'final_answer':m})
    def research_agent(self,plan: str):
        """The plan to be executed
        
        Args:
            plan: The plan to be executed        
        """

        research_agent = ResearchAgent()
        research_agent.add_message("user",plan)
        return research_agent

    @agent_tool
    def judge_agent(self,final_answer: str):
        """Judge tool that will evaluate the answer from the research agent. It should be used after the research agent have finished its job.

        Args:
            final_answer: The final answer to be judged

        """ 
        judge_agent = JudgeAgent(user_query = self.user_query)
        prompt = f"Initial user question:{self.user_query}\n\nThe answer from the research agent:{final_answer}"
        judge_agent.add_message("user",prompt)
        return judge_agent

In [17]:
main_agent = MainAgent()
main_agent.add_message("user","Can you find 5 papers on quantum machine learning?")

In [18]:
response_iterator = await main_agent.run(max_retries = 10)
async for response in response_iterator:
    continue

Executing agent: MainAgent
Executing agent: planing_agent
Executing tool: task_completed (agent: planing_agent)
Executing agent: research_agent
Executing tool: search_papers (agent: research_agent)
Executing agent: research_agent


In [21]:
for m in main_agent.conv_history.messages:
    print(m)
    print("--------------------------------")

role='system' agent_name='MainAgent' content=[Content(type='text', text='You are an experienced scientific researcher.\nYour goal is to help the user with their scientific research.\n\nBased on the user query, decide if you need to perform a research or if you can answer the question directly.\n- You should perform a research if the user query requires any supporting evidence or information. In this case, you should call first the planning agent to create a plan and then the research agent to execute the plan, and finally the judge agent to evaluate the final answer.\n- You should answer the question directly only for simple conversational questions, like "how are you?".\n')] tool_calls=None tool_call_id=None tool_name=None hidden=False id=None usage=None timestamp=datetime.datetime(2025, 9, 27, 9, 24, 20, 700055)
--------------------------------
role='user' agent_name='MainAgent' content=[Content(type='text', text='Can you find 5 papers on quantum machine learning?')] tool_calls=None 

In [22]:
for m in main_agent.child_agents[1].conv_history.messages:
    print(m)
    print("--------------------------------")

role='system' agent_name='research_agent' content=[Content(type='text', text='# IDENTITY AND PURPOSE\n\nYou are an experienced scientific researcher. \nYour goal is to help the user with their scientific research. You have access to a set of external tools to complete your tasks.\nFollow the plan given by the planning agent to successfully complete the task.\nAt the end call the complete_task tool to provide the final answer to the user.\n\nAdd extensive inline citations to support any claim made in the answer.\n\n\n# EXTERNAL KNOWLEDGE\n\n## CORE API\n\nThe CORE API has a specific query language that allows you to explore a vast papers collection and perform complex queries. See the following table for a list of available operators:\n\n| Operator       | Accepted symbols         | Meaning                                                                                      |\n|---------------|-------------------------|--------------------------------------------------------------------

In [ ]:
import gradio as gr
from litellm import ModelResponseStream
from agente.models.schemas import StreamResponse


def get_new_agent():
    """Create a fresh agent instance"""
    print("Creating a new agent")
    new_agent = MainAgent()
    return new_agent

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    # Initialize with a function call instead of direct instantiation
    main_agent = gr.State(value=None)

    def user(user_message, history,agent):
        if agent is None:
            agent = get_new_agent()
        agent.add_message("user", user_message)
        history.append({"role": "user", "content": user_message})
        return "", history, agent

    async def bot(history, agent):
        if agent is None:
            agent = get_new_agent()
        if not history:
            yield [], agent
            return
        
        history.append({"role": "assistant", "content": ""})
        # agent_name = ""
        # async for chunk in agent.run(max_retries=10):            
        #     if chunk.content:
        #         if hasattr(chunk, "is_tool_call"):
        #             if chunk.is_tool_call:
        #                 tool_name = chunk.tool_name
        #                 temp = f"Calling the {tool_name} tool..."
        #                 if history[-1]["content"].endswith(temp):
        #                     pass
        #                 else:
        #                     history[-1]["content"] += f"\n\n{temp}"
        #             else:
        #                 history[-1]["content"] += chunk.content
        #             yield history, agent
        #         else:
        #             history[-1]["content"] += chunk.content
        #             yield history, agent

        response_generator = await agent.run(stream=True)
        async for chunk in response_generator:
            #check if chunk is litellm or agente
            if isinstance(chunk, ModelResponseStream):
                if chunk.choices[0].delta.content:
                    history[-1]["content"] += chunk.choices[0].delta.content
                    yield history, agent
            elif isinstance(chunk, StreamResponse):
                if chunk.content:
                    history[-1]["content"] += chunk.content
                    yield history, agent
            else:
                raise ValueError(f"Unknown chunk type: {type(chunk)}")


    def reset_state():
        return None, get_new_agent()

    msg.submit(user, [msg, chatbot,main_agent], [msg, chatbot, main_agent]).then(
        bot, [chatbot, main_agent], [chatbot, main_agent]
    )
    clear.click(reset_state, None, [chatbot, main_agent])

demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Creating a new agent
Executing agent: MainAgent
Executing agent: MainAgent
Executing agent: planing_agent
Executing tool: task_completed (agent: planing_agent)
Executing agent: research_agent
Executing tool: search_papers (agent: research_agent)
Executing agent: research_agent
